# OCR

## With Unstructured.io

Hacer las peticiones al `http://localhost:8000`

Con el siguente comando se puede correr el contenedor de UNSTRUCTURED (con la opcion de `--rm` para que se elimine el contenedor al detenerlo):

```bash
docker run -p 8000:8000 -d --rm --name unstructured-api downloads.unstructured.io/unstructured-io/unstructured-api:latest --port 8000 --host 0.0.0.0
```

_Sin la opcion `--rm`:_
```bash
docker run -p 8000:8000 -d --name unstructured-api downloads.unstructured.io/unstructured-io/unstructured-api:latest --port 8000 --host 0.0.0.0
```

*Más imformación sobre la imagen: ([UnstructuredLoader | 🦜️🔗 Langchain](https://js.langchain.com/v0.2/docs/integrations/document_loaders/file_loaders/unstructured/))*

*Más imformación sobre el SDK de Python: [Process an individual file by using the Unstructured Python SDK - Unstructured](https://docs.unstructured.io/api-reference/api-services/sdk-python)*

In [ ]:
# Dependencies for unstructured

%pip install unstructured-client

Prueba con el siguiente bloque de codigo que hace la peticion al contenedor de UNSTRUCTURED, el cual se encarga de hacer el OCR a un archivo PDF; para despues guardar la salida en un archivo de texto.

```python
import os
import json
import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    server_url="http://localhost:8000",
)

filename = "file.pdf"
with open(filename, "rb") as f:
    data = f.read()

req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=shared.Files(
            content=data,
            file_name=filename,
        ),
        strategy=shared.Strategy.HI_RES,
        languages=['eng'],
        split_pdf_page=True,            # If True, splits the PDF file into smaller chunks of pages.
        split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
        split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
    ),
)

try:
    res = client.general.partition(request=req)
    element_dicts = [element for element in res.elements]
    json_elements = json.dumps(element_dicts, indent=2)

    # Print the processed data.
    print(json_elements)

    # Write the processed data to a local file.
    with open("salida.txt", "w") as file:
        file.write(json_elements)
except Exception as e:
    print(e)
```

### Clase OCR

En la clase `OCR` se encuentra el metodo `get_ocr()` que en este caso se encarga de hacer el OCR a archivos de tipo PDF o rtf.
Tambien se cuenta con el metodo `get_dev_ocr()` que es el encargado de hacer la extracción de texto de archivos de texto plano.

**IMPORTANTE:**
Hay problemas con la concurrencia

In [ ]:
import json
from pathlib import Path
from typing import List, Dict
from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared


class OCR:
    def __init__(self):
        self.unstructured_client = UnstructuredClient(server_url="http://localhost:8000")

    def get_ocr(self, file_path: str) -> List[Dict]:
        with open(file_path, "rb") as f:
            data = f.read()

        req = operations.PartitionRequest(
            partition_parameters=shared.PartitionParameters(
                files=shared.Files(
                    content=data,
                    file_name=file_path,
                ),
                strategy=shared.Strategy.AUTO,
                languages=['eng', 'spa'],
                split_pdf_page=False,            # If True, splits the PDF file into smaller chunks of pages.
                # split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
                # split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
            ),
        )
        try:
            res = self.unstructured_client.general.partition(request=req)
            element_dicts = [element for element in res.elements]
            json_elements = json.dumps(element_dicts, indent=2)

            # Print the processed data.
            print(json_elements)
            return element_dicts
        except Exception as e:
            print(e)

    def get_dev_ocr(self, file_path: str) -> Dict:
        file = Path(file_path)
        metadata = {"filetype": f'text/{file.suffix[1:]}' , "filename": file.name}
        text = file.read_text(encoding='utf-8')
        data = {
            'metadata': metadata, 
            'text': text
        }
        return [data]


---

## Without Unstructured.io

Unstructured.io es una herramienta poderosa, pero aparte de que tiene opciones de pago para mejor rendimiento, es bastante tardado hacer el OCR de documentos con elementos complejos (como imagenes, tablas, listas, etc.). Por lo que se encontro una alternativa que puede ser la libreria de ["pymupdf/PyMuPDF: PyMuPDF is a high performance Python library for data extraction, analysis, conversion & manipulation of PDF (and other) documents."](https://github.com/pymupdf/PyMuPDF) para hacer la extracción de texto a archivos PDF.

Los archivos PDF generalmente contienen imagenes, tablas, listas, etc. Por lo que primero se puede aplicar una capa de OCR al documento, para que la extracción de texto sea mas precisa como con un OCR de Unstructured.

Para agregar la capa de OCR a un archivo PDF se puede hacer con la herramienta de ["ocrmypdf/OCRmyPDF: OCRmyPDF adds an OCR text layer to scanned PDF files, allowing them to be searched"](https://github.com/ocrmypdf/OCRmyPDF).
OCRmyPDF ocupa la libreria de ["tesseract-ocr/tesseract: Tesseract Open Source OCR Engine"](https://github.com/tesseract-ocr/tesseract) para hacer el OCR a los archivos PDF. Por lo que primero se debe instalar la libreria de tesseract-ocr y despues instalar OCRmyPDF.

Como OCRmyPDF es una herramienta de linea de comandos, se puede hacer uso de la libreria de `subprocess` para hacer uso de OCRmyPDF en Python.

```python
import subprocess
import os

def ocr_pdf(input_pdf, output_pdf, language='eng+spa'):
    """
    Aplica OCR a un archivo PDF usando OCRmyPDF.

    :param input_pdf: Ruta al archivo PDF de entrada.
    :param output_pdf: Ruta al archivo PDF de salida.
    :param language: Idioma para OCR (por defecto 'eng+spa' para español).
    """
    try:
        # Construir el comando
        comando = [
            'ocrmypdf',
            '-l', language,
            '--force-ocr',
            '--jobs', '6',
            '--output-type', 'pdf',
            input_pdf,
            output_pdf
        ]

        # Ejecutar el comando
        subprocess.run(comando, check=True)
        print(f"OCR aplicado exitosamente a {input_pdf}. Salida: {output_pdf}")
    except subprocess.CalledProcessError as e:
        print(f"Error al aplicar OCR: {e}")

# Uso del script
if __name__ == "__main__":
    ruta_input = "ruta al archivo PDF de entrada"
    ruta_output = "ruta al archivo PDF de salida (puede ser el mismo archivo)"
    
    # Verificar si el archivo de entrada existe
    if os.path.exists(ruta_input):
        ocr_pdf(ruta_input, ruta_output)
    else:
        print(f"El archivo {ruta_input} no existe.")
```

### Clase OCR

En la clase `OCR` se encuentra el metodo `get_ocr()` que en este caso se encarga de hacer la extracción de texto de archivos PDF con la capa de OCR aplicada.
Este metodo manda llamar a la funcion `ocr_pdf()` que se encarga de aplicar la capa de OCR a un archivo PDF.
Tambien se cuenta con el mismo metodo `get_dev_ocr()` que es el encargado de hacer la extracción de texto de archivos de texto plano.

In [ ]:
# Dependencies for PyMuPDF

%pip install PyMuPDF

In [2]:
"""
This module provides functionalities for Optical Character Recognition (OCR).
"""

import subprocess
from pathlib import Path
from typing import List, Dict, Union
import pymupdf


class OCR:
    @staticmethod
    def _ocr_pdf(input_pdf: Union[str, Path], output_pdf: Union[str, Path], language='eng+spa') -> None:
        """
        Adds an OCR text layer to scanned PDF files, allowing them to be searched using OCRmyPDF.

        Args: 
            input_pdf (str, Path): The path to the input PDF file.
            output_pdf (str, Path): The path to the output PDF file.
            language (str): The language(s) to use for OCR. Default is 'eng+spa' (English and Spanish).
        
        Returns:
            None
        """
        try:
            # Construir el comando
            comando = [
                'ocrmypdf',
                '-l', language,
                '--force-ocr',
                '--jobs', '6',  # Número de trabajos en paralelo
                '--output-type', 'pdf',
                str(input_pdf),
                str(output_pdf)
            ]

            # Ejecutar el comando
            subprocess.run(comando, check=True)
            print(f"OCR aplicado exitosamente a {input_pdf}. Salida: {output_pdf}")
        except subprocess.CalledProcessError as e:
            print(f"Error al aplicar OCR: {e}")

    @classmethod
    def get_ocr(cls, file_path: str) -> List[Dict]:
        """
        Extracts text of each page from a PDF file using PyMuPDF.

        Args:
            file_path (str): The path to the PDF file.
        
        Returns:
            List[Dict]: A list of dictionaries containing the text and metadata of each page.
        """
        file = Path(file_path).resolve()
        cls._ocr_pdf(file, file)
        elements = []
        metadata = {
            'filetype': 'application/pdf',
            'filename': file.name,
            'page_number': 0
        }
        doc = pymupdf.open(file)  # Abrir el archivo PDF
        for page in doc:
            metadata_copy = metadata.copy()  # Crear una copia del diccionario
            metadata_copy['page_number'] = page.number + 1
            elements.append({
                'metadata': metadata_copy,
                'text': page.get_text().encode('utf-8')
            })
        return elements
    
    @staticmethod
    def get_dev_ocr(file_path: str) -> List[Dict]:
        """
        Extracts text from a plain text file, e.g. {'.txt', '.html', '.py'}.

        Args:
            file_path (str): The path to the text file.

        Returns:
            List[Dict]: A list of dictionaries containing the text and metadata of the file.
        """
        file = Path(file_path)
        metadata = {"filetype": f'text/{file.suffix[1:]}' , "filename": file.name}
        text = file.read_text(encoding='utf-8')
        data = {
            'metadata': metadata, 
            'text': text
        }
        return [data]

#### Ejemplo de uso

In [5]:
from pprint import pprint

data = OCR.get_dev_ocr("example.txt")
pprint(data)

print("\n\n----------------------------------------\n\n")

data = OCR.get_ocr("example.pdf")
pprint(data)

[{'metadata': {'filename': 'example.txt', 'filetype': 'text/txt'},
  'text': 'Los perros son animales domésticos que han acompañado a los seres '
          'humanos desde hace miles de años. Son miembros de la familia de los '
          '*cánidos*, parientes de lobos, coyotes y zorros, pero gracias a la '
          'domesticación se han adaptado para convivir y trabajar junto a las '
          'personas. Hoy en día, son una de las mascotas más populares y '
          'queridas en todo el mundo.\n'
          '\n'
          '### Características generales\n'
          '\n'
          'Los perros tienen un sentido del olfato extremadamente '
          'desarrollado, de hecho, es mucho más potente que el de los humanos, '
          'lo que les permite identificar aromas con una precisión '
          'impresionante. Esto los hace ideales para tareas como la detección '
          'de sustancias, el rastreo en operaciones de rescate y el trabajo de '
          'apoyo en seguridad. Además, son a

This PDF is marked as a Tagged PDF. This often indicates that the PDF was generated from an office document and does not need OCR. PDF pages processed by OCRmyPDF may not be tagged correctly.
Start processing 2 pages concurrently
    1 page already has text! - rasterizing text and running OCR anyway
    2 page already has text! - rasterizing text and running OCR anyway
Postprocessing...


OCR aplicado exitosamente a /Users/jorge-jrzz/Desktop/chunks/API/dev/example.pdf. Salida: /Users/jorge-jrzz/Desktop/chunks/API/dev/example.pdf
[{'metadata': {'filename': 'example.pdf',
               'filetype': 'application/pdf',
               'page_number': 1},
  'text': b'Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de '
          b'cualquier marcador de posici\xc3\xb3n (como este, por\nejemplo) y c'
          b'omience a escribir.\nPara aplicar facilmente cualquier formato de'
          b' texto que vea en esta pagina, vaya al grupo\nEstilos, que encont'
          b'rara en la pestana Inicio de la cinta de opciones.\n\xc2\xa2 Desea '
          b'insertar una imagen de sus archivos o agregar una forma o un cua'
          b'dro de texto?\n\xc2\xa1Adelante! En la pesta\xc3\xb1a Insertar de'
          b' la cinta de opciones, pulse la opci\xc3\xb3n que necesite.\n'},
 {'metadata': {'filename': 'example.pdf',
               'filetype': 'application/pdf',
             

Image optimization did not improve the file - optimizations will not be used
Image optimization ratio: 1.00 savings: -0.0%
Total file size ratio: 0.26 savings: -284.2%
